# Determining the CDS Basket Spread

An analysis of pricing a CDS Basket 

## THIS IS BROKEN - TO BE REPAIRED

In [1]:
# Move into the local folder of notebook
%pwd
%cd notebooks/products/credit/
%pwd

[WinError 3] The system cannot find the path specified: 'notebooks/products/credit/'
C:\Users\Dominic\Dropbox\Desktop\RESEARCH_DB\FinancePy\Code\financepy-git\notebooks\products\credit


'C:\\Users\\Dominic\\Dropbox\\Desktop\\RESEARCH_DB\\FinancePy\\Code\\financepy-git\\notebooks\\products\\credit'

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from financepy.products.credit import *
from financepy.products.rates import *
from financepy.utils import *

In [4]:
value_dt = Date(1, 8, 2007)
settle_dt = value_dt.add_weekdays(1)

## Build Ibor Curve

In [5]:
dc_type = DayCountTypes.THIRTY_E_360_ISDA
fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap_type = SwapTypes.PAY
depo = IborDeposit(settle_dt, "1D", 0.0502, dc_type)
depos = [depo]
swap1 = IborSwap(settle_dt,"1Y",swap_type,0.0502,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"2Y",swap_type,0.0502,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"3Y",swap_type,0.0501,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"4Y",swap_type,0.0502,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"5Y",swap_type,0.0501,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [6]:
step_in_dt = value_dt.add_weekdays(0)
value_dt = step_in_dt

In [7]:
maturity_3yr = value_dt.next_cds_date(36)
maturity_5yr = value_dt.next_cds_date(60)
maturity_7yr = value_dt.next_cds_date(84)
maturity_10yr = value_dt.next_cds_date(120)

### Heterogeneous Curves

In [8]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_dt, maturity_3yr, spd3Y)
    cds5Y = CDS(step_in_dt, maturity_5yr, spd5Y)
    cds7Y = CDS(step_in_dt, maturity_7yr, spd7Y)
    cds10Y = CDS(step_in_dt, maturity_10yr, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [9]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40

In [10]:
cdsIndex = CDSIndexPortfolio()

In [11]:
spd3Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_3yr, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_5yr, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_7yr, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_10yr, heteroIssuerCurves)

In [12]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.678820972252527
Homogeneous curve 5Y: 35.53919235929798
Homogeneous curve 7Y: 49.01185349544258
Homogeneous curve 10Y: 61.41369871102577


In [13]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_dt, maturity_3yr, spd3Y)
    cds5Y = CDS(step_in_dt, maturity_5yr, spd5Y)
    cds7Y = CDS(step_in_dt, maturity_7yr, spd7Y)
    cds10Y = CDS(step_in_dt, maturity_10yr, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Baskets

In [14]:
num_credits = 5

In [15]:
issuer_curves = heteroIssuerCurves[0:num_credits]

In [16]:
basketMaturity = Date(20, 12, 2011)
cdsIndex = CDSIndexPortfolio()

In [17]:
intrinsicSpd = cdsIndex.intrinsic_spread(value_dt, step_in_dt, basketMaturity, issuer_curves) * 10000.0

print("INTRINSIC SPD BASKET MATURITY", intrinsicSpd)

totalSpd = cdsIndex.total_spread(value_dt, step_in_dt, basketMaturity, issuer_curves) * 10000.0

print("SUMMED UP SPD BASKET MATURITY", totalSpd)

minSpd = cdsIndex.min_spread(value_dt, step_in_dt, basketMaturity, issuer_curves) * 10000.0

print("MINIMUM SPD BASKET MATURITY", minSpd)

maxSpd = cdsIndex.max_spread(value_dt, step_in_dt, basketMaturity, issuer_curves) * 10000.0

print("MAXIMUM SPD BASKET MATURITY", maxSpd)

INTRINSIC SPD BASKET MATURITY 29.03716340808204
SUMMED UP SPD BASKET MATURITY 145.80584474439158
MINIMUM SPD BASKET MATURITY 9.620144164866993
MAXIMUM SPD BASKET MATURITY 73.20754781040397


In [18]:
basket = CDSBasket(value_dt,basketMaturity)

## Gaussian Copula Model

THERE IS AN ERROR HERE THAT NEEDS TO BE FIXED

In [19]:
seed = 42
doF = 5
num_trials = 5000

print("NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10")
print("=========================================================")

for ntd in range(1, num_credits + 1):
    for beta in [0.0, 0.25, 0.5, 0.75, 0.90, 0.9999]:
        rho = beta * beta
        beta_vector = np.ones(num_credits) * beta
        corr_matrix = corr_matrix_generator(rho, num_credits)
        for num_trials in [5000]:
            v1 = basket.value_gaussian_mc(value_dt,ntd,issuer_curves,corr_matrix,libor_curve,num_trials,seed)
            v2 = basket.value_1f_gaussian_homo(value_dt,ntd,issuer_curves,beta_vector,libor_curve)
            v3 = basket.value_student_t_mc(value_dt, ntd, issuer_curves, corr_matrix, doF, libor_curve, num_trials,seed)
            print("%7d  %5.2f    %d    %9.3f   %9.3f    %9.3f"% (num_trials, rho, ntd, v1[2] * 10000, v2[3] * 10000, v3[2] *10000))
    print("=========================================================")


NTrials   Rho    NTD    SPD_GC_MC    SPD_1FGC    SPD_ST10


   5000   0.00    1      146.142     146.566      133.073


   5000   0.06    1      141.088     144.467      129.984


   5000   0.25    1      130.670     135.682      123.001


   5000   0.56    1      112.982     113.376      103.433


...

   5000   0.06    5        0.000       0.000        0.000


   5000   0.25    5        0.000       0.008        0.257


   5000   0.56    5        0.648       0.441        2.572


   5000   0.81    5        3.122       2.944        4.804


   5000   1.00    5       10.413       9.778        9.704


Copyright (c) 2020 Dominic O'Kane